In [1]:
import json

train_json = "/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/train_textlines_others_add_text_400_converted_add_in_graphical_others_grouped.json"

train_annotations = json.load(open(train_json))

In [2]:
train_annotations['categories']

[{'id': 1, 'name': 'Picture', 'supercategory': 'super fig'},
 {'id': 2, 'name': 'Table', 'supercategory': 'super tab'},
 {'id': 3, 'name': 'Formula', 'supercategory': 'super formula'},
 {'id': 4, 'name': 'Paragraph', 'supercategory': 'super para'},
 {'id': 5, 'name': 'other', 'supercategory': 'super other'}]

In [3]:
'''
Conversation Format:

{
    "image": "image_path",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nPOD: "
        },
        {
            "from": "gpt",
            "value": "[Box1] [Logical Role]\n[Box2] [Logical Role]\n..."
        }
    ]
}
'''

conversations = []


'\nConversation Format:\n\n{\n    "image": "image_path",\n    "conversations": [\n        {\n            "from": "human",\n            "value": "<image>\nPOD: "\n        },\n        {\n            "from": "gpt",\n            "value": "[Box1] [Logical Role]\n[Box2] [Logical Role]\n..."\n        }\n    ]\n}\n'

In [4]:
tags = set([anno['tags'][0] for anno in train_annotations['annotations']])
print(tags)

{'Formula', 'Table', 'Page-footer', 'others', 'Text', 'Picture', 'Footnote', 'Title', 'Page-header', 'List-item', 'Section-header', 'Caption'}


In [5]:
imageid2annotations = {}

for annotation in train_annotations["annotations"]:
    if annotation["image_id"] not in imageid2annotations:
        imageid2annotations[annotation["image_id"]] = []
    imageid2annotations[annotation["image_id"]].append(annotation)

imageid2image = {}
for image in train_annotations["images"]:
    imageid2image[image["id"]] = image

conversations = []
for image_id, annotations in imageid2annotations.items():
    image = imageid2image[image_id]
    image_path = image["file_name"]
    conversation = {
        "image": "train/" + image_path,
        "conversations": []
    }
    conversation["conversations"].append({
        "from": "human",
        "value": "<image>\nPOD: "
    })
    gt_text = ""
    gt_boxes = []
    gt_logical_roles = []
    for annotation in annotations:
        bbox = annotation["quad_coord"][:2] + annotation["quad_coord"][4:6]
        # x = int(x/w * 1000)
        # y = int(y/h * 1000)
        bbox = [int(bbox[0] / image["width"] * 1000), int(bbox[1] / image["height"] * 1000), int(bbox[2] / image["width"] * 1000), int(bbox[3] / image["height"] * 1000)]
        if "others" in annotation["tags"]:
            # exclude others
            continue
        gt_boxes.append(bbox)
        gt_logical_roles.append(annotation["tags"][0])

    # sort the boxes from top to bottom from left to right
    # 将 gt_boxes 和 gt_logical_roles 结合在一起，根据 gt_boxes 进行排序
    sorted_pairs = sorted(zip(gt_boxes, gt_logical_roles), key=lambda x: (x[0][1], x[0][0]))
    if len(sorted_pairs) == 0:
        continue
    # 解压缩，提取排序后的 gt_boxes 和 gt_logical_roles
    gt_boxes, gt_logical_roles = zip(*sorted_pairs)

    # 将它们转换回列表（因为 zip 返回的是元组）
    gt_boxes = list(gt_boxes)
    gt_logical_roles = list(gt_logical_roles)
    
    for bbox, logical_role in zip(gt_boxes, gt_logical_roles):
        gt_text += f"{str(bbox)} {logical_role}\n"
    
    conversation["conversations"].append({
        "from": "gpt",
        "value": gt_text
    })
    conversations.append(conversation)

In [1]:
conversations[0]

NameError: name 'conversations' is not defined

In [7]:
json.dump(conversations, open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/got_annotations/train_got_conversations_wo_others.json", "w"), indent=4)

In [7]:
# 可视化GT
import json

conversations = json.load(open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/got_annotations/train_got_conversations_wo_others.json"))
converstation = conversations[0]
image_path = "/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/" + converstation["image"]
outputs = converstation["conversations"][1]["value"]
print(outputs)
outputs_list = outputs.split('\n')
# visualize the bboxes and logical role in args.image_file: [117, 88, 328, 100] ['Section-header']
import cv2
image_viz = cv2.imread(image_path)
w, h = image_viz.shape[1], image_viz.shape[0]
for out in outputs_list:
    try:
        bbox, logical_role = out.split('] ')
    except:
        continue
    bbox = bbox[1:].split(', ')
    # bbox = [int(i) for i in bbox]
    bbox = [int(i) / 1000 for i in bbox]
    bbox = [int(bbox[0]*w), int(bbox[1]*h), int(bbox[2]*w), int(bbox[3]*h)]
    if bbox[0] < 0 or bbox[1] < 0 or bbox[2] < 0 or bbox[3] < 0:
        continue
    image_viz = cv2.rectangle(image_viz, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    image_viz = cv2.putText(image_viz, logical_role, (bbox[0], bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imwrite('test_gt.jpg', image_viz)

[70, 54, 433, 74] Page-header
[97, 114, 867, 155] Text
[97, 162, 866, 217] Text
[97, 223, 861, 278] Text
[97, 285, 813, 326] Text
[97, 333, 836, 360] Text
[94, 361, 891, 473] Table
[97, 488, 876, 529] Text
[97, 549, 789, 567] Section-header
[97, 572, 608, 587] Text
[96, 592, 886, 675] Table
[97, 687, 886, 742] Text
[97, 749, 782, 776] Text
[97, 783, 887, 838] Text
[924, 963, 947, 979] Page-footer



True

In [1]:
import json

val_data = json.load(open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/val_textlines_others_add_text_new_coco.json"))

In [3]:
len(val_data['annotations'])

328279

In [4]:
val_data['annotations'] = [anno for anno in val_data['annotations'] if anno['category_id'] != 12]

In [5]:
val_data['annotations'].__len__()

99816

In [6]:
json.dump(val_data, open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/val_textlines_others_add_text_new_coco_wo_others.json", "w"), indent=4)

In [1]:
# construct data for localization and recognition concurrently

import json

train_json = "/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/train_textlines_others_add_text_400_converted_add_in_graphical_others_grouped.json"

train_annotations = json.load(open(train_json))

In [5]:
imageid2annotations = {}

for annotation in train_annotations["annotations"]:
    if annotation["image_id"] not in imageid2annotations:
        imageid2annotations[annotation["image_id"]] = []
    imageid2annotations[annotation["image_id"]].append(annotation)

imageid2image = {}
for image in train_annotations["images"]:
    imageid2image[image["id"]] = image

conversations = []
for image_id, annotations in imageid2annotations.items():
    image = imageid2image[image_id]
    image_path = image["file_name"]
    conversation = {
        "image": "train/" + image_path,
        "conversations": []
    }
    conversation["conversations"].append({
        "from": "human",
        "value": "<image>\nPOD and OCR: "
    })
    gt_text = ""
    gt_boxes = []
    gt_logical_roles = []
    gt_texts = []
    for annotation in annotations:
        bbox = annotation["quad_coord"][:2] + annotation["quad_coord"][4:6]
        # x = int(x/w * 1000)
        # y = int(y/h * 1000)
        bbox = [int(bbox[0] / image["width"] * 1000), int(bbox[1] / image["height"] * 1000), int(bbox[2] / image["width"] * 1000), int(bbox[3] / image["height"] * 1000)]
        if "others" in annotation["tags"]:
            # exclude others
            continue
        gt_boxes.append(bbox)
        gt_logical_roles.append(annotation["tags"][0])
        text_contents = ' '.join(annotation["textline_contents"])
        if text_contents:
            gt_texts.append(text_contents)
        else:
            gt_texts.append(annotation["tags"][0])

    # sort the boxes from top to bottom from left to right
    # 将 gt_boxes 和 gt_logical_roles 结合在一起，根据 gt_boxes 进行排序
    sorted_pairs = sorted(zip(gt_boxes, gt_logical_roles, gt_texts), key=lambda x: (x[0][1], x[0][0]))
    if len(sorted_pairs) == 0:
        continue
    # 解压缩，提取排序后的 gt_boxes 和 gt_logical_roles
    gt_boxes, gt_logical_roles, gt_texts = zip(*sorted_pairs)

    # 将它们转换回列表（因为 zip 返回的是元组）
    gt_boxes = list(gt_boxes)
    gt_logical_roles = list(gt_logical_roles)
    gt_texts = list(gt_texts)
    
    for bbox, logical_role, text in zip(gt_boxes, gt_logical_roles, gt_texts):
        gt_text += f"{str(bbox)} {logical_role} <content>{text}</content>\n"
    
    conversation["conversations"].append({
        "from": "gpt",
        "value": gt_text
    })
    conversations.append(conversation)

In [6]:
conversations[0]

{'image': 'train/c6effb847ae7e4a80431696984fa90c98bb08c266481b9a03842422459c43bdd.png',
 'conversations': [{'from': 'human', 'value': '<image>\nPOD and OCR: '},
  {'from': 'gpt',
   'value': '[70, 54, 433, 74] Page-header <content>NOTES TO THE FINANCIAL STATEMENTS</content>\n[97, 114, 867, 155] Text <content>Finance receivables that originated outside the U.S. were $52.7 billion and $47.5 billion at December 31, 2004 and 2003, respectively. Other finance receivables consisted primarily of real estate, commercial and other collateralized loans and accrued interest. </content>\n[97, 162, 866, 217] Text <content>Included in net finance and other receivables at December 31, 2004 and 2003 were $16.9 billion and $14.3 billion, respectively, of receivables that have been sold for legal purposes to consolidated securitization SPEs and are available only for repayment of debt issued by those entities, and to pay other securitization investors and other participants; they are not available to pa

In [7]:
json.dump(conversations, open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/got_annotations/train_got_conversations_wo_others_pod_and_ocr.json", "w"), indent=4)